# **Predicting Stress Level By Training a Multilayer Perceptron**

This machine learning project aims to predict the stress level being experienced by an individual given their sleep health and various lifestyle habits. The project uses a Multilayer Perceptron (MLP) as the machine learning algorithm. [Sleep Health Data](https://www.kaggle.com/datasets/imaginativecoder/sleep-health-data-sampledt) is the dataset used for the project, and is acquired from Kaggle. 

This is a course requirement for CS 180 (Artificial Intelligence) Course of the Department of Computer Science, College of Engineering, University of the Philippines, Diliman under the guidance of Carlo Raquel for A.Y. 2023-2024.

- MAXIMO, Calvin James T.
- MENDOZA, Janelle M.
- MURILLO, Joana Marie V.

The GitHub repository for this project can be accessed [here](https://github.com/cjmax34/cs180-project).

## 1. Importing libraries

First, we have to import the libraries needed for the project. The libraries imported here are for loading the dataset, and performing exploratory data analysis (EDA). Additional libraries for modeling, and metrics will be imported later in the notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Loading the dataset

Next, we will now load the dataset and store it in a variable named `dataset`.

In [ ]:
dataset = pd.read_csv('Sleep_Data_Sampled.csv')

## 3. Performing Exploratory Data Analysis (EDA)

Now we will perform EDA on our dataset. This is important to help us understand the dataset more before we make assumptions. There might be patterns, trends, and relationships in the dataset that may not be visible at first glance. There might also be rows that contain missing values, outliers, inconsistencies, or biases, which could lead to inaccurate results. By gaining a deeper understanding of the dataset at hand through performing EDA, we are able to choose the appropriate techniques and approaches in training the model.

### Data exploration

#### Shape of dataset

We want to know the shape of the dataset, or the number of rows and columns it has. We can easily do this by calling `shape()` on `dataset`. 

In [ ]:
# Prints (# of rows, # of columns)
dataset.shape

Our dataset has 15000 rows (!!) and 13 columns/features.

#### Getting information about the dataset

We want more information about the dataset. We can do this by calling `info()` on `dataset`. This provides us essential information such as the number of columns, column names, and data types of each column.

In [ ]:
# Provides information about the dataset
dataset.info(); 

In [ ]:
# Print column names of dataset
dataset.columns

We can see that there are 13 columns in total, namely: Person ID, Gender, Age, Occupation, Sleep Duration, Quality of Sleep, Physical Activity Level, Stress Level, BMI Category, Blood Pressure, Heart Rate, Daily Steps, and Sleep Disorder. 

The data types across all columns are float, int, and object (string).

#### Unique values per column

In this part, we want to identify the unique values for each of the dataset's columns/features. This helps us ensure the consistent formatting of the values across all columms and assess the complexity of each categorical feature (number of unique values).

In [ ]:
# Number of unique values per column
dataset.nunique()

The number of unique Person IDs is 15000, which makes sense because it is used to identify an individual in the dataset. We will deal with this later.

In [ ]:
# Unique values per categorical feature
for col in dataset.select_dtypes(include='object').columns:
    print(f"{col}: {list(dataset[col].unique())}")

From the output above, it seems that the formatting of each column/feature's values is consistent. However, we have to preprocess some of the values, such as "Male" and "Female" in the Gender category, "Normal Weight" in the BMI Category feature, and the systolic and diastolic blood pressure measurements in the Blood Pressure category. We will delve into this later on.

#### Statistical summary

The `describe()` method returns a statistical summary of the numerical features in the given dataset. For each column, it returns these information.

count - The number of not-empty values.\
mean - The average (mean) value.\
std - The standard deviation.\
min - the minimum value.\
25% - The 25% percentile.\
50% - The 50% percentile.\
75% - The 75% percentile.\
max - the maximum value.

Reference: https://www.w3schools.com/python/pandas/ref_df_describe.asp

In [ ]:
dataset.describe()

We can see from the table above that all numerical features have the same count (15000).

The average age is 44.13 years. The average sleep duration is 7 hours. The average quality of sleep (1-10, 1 being the lowest and 10 being the highest) is 7.13. The average amount of daily physical activity is 59.93 minutes, or nearly an hour. The average stress level (1-10, 1 being the lowest and 10 being the highest) is 5.65. The average heart rate is 70.86 beats per minute. The average number of daily steps is 6795.

#### Checking for the presence of null values

An important part of data exploration is checking for the presence of null values in the dataset. It is vital to handle null values because they can produce inaccurate or misleading results. Some machine learning algorithms, especially MLP, also cannot handle null values. Ultimately, it helps us in deciding the best way to handle the null values through various methods such as imputation and removal of rows.

In [ ]:
# Checking for null values
dataset.isnull().sum()

Fortunately, there are no null values across **ALL** rows of the dataset.

### Data visualization

Data visualization is another important part of every machine learning project. In this part, we will be using graphs to better understand the patterns and trends in the data. These patterns and trends can easily be understood through visualization especially if they are hard to discern from the raw data. It condenses quite complex information into an easily digestible format. We can also discover what features correlate strongly with the target variable (stress level). Ultimately, it is a powerful tool for effectively communicating insights on data to a wider audience.

#### Distribution of stress level

We want to visualize the distribution of the stress level feature, which is our target variable, in our dataset. The code block below is from [Tanaya Tipre's project](https://www.kaggle.com/code/tanayatipre/stress-level-detection#5.-Data-Visualization), with the labels modified for clarity.

In [ ]:
# Distribution of stress level
sns.countplot(x='Stress Level', data=dataset)

plt.xlabel('Stress Level')

plt.ylabel('Count')

plt.title('Distribution of Stress Level')

# Displaying the plot
plt.show()

From the figure above, we can observe that the stress level is not distributed evenly, and a stress level of 6 is found across many records in the dataset.

#### Distribution of numerical features

We want to visualize the distribution of the numerical features of the dataset. We might be able to identify outliers and potential patterns across these features.

In [ ]:
# List of features to plot histograms for
features = [col for col in dataset.columns if col not in ['Stress Level', 'Person ID']]

# Plot histograms for each feature
dataset.hist(column=features, bins=10, figsize=(10, 10))
plt.suptitle("Histograms of Features", fontsize=20)
plt.show()

Insert explanation here...

## 4. Data preprocessing

Now that we have uncovered the possible trends and patterns in the dataset, we will now perform preprocessing. This involves renaming of columns or column values (if applicable), dropping of unused features, normalization of numerical features, label encoding (converting categorical data to numerical data), and dealing with null values.

### Importing libraries for preprocessing

In [ ]:
# Importing the necessary libraries to conduct preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()  # Instantiate the label encoder

There are categorical features in the dataset such as `Gender`, `Occupation`, `BMI Category`, `Sleep Disorder`, and also the target variable `Stress Level`. Label encoding is utilized here to convert categorical values into numerical values by assigning a unique numerical label for each categorical value. This is done by importing `LabelEncoder`.

On the other hand, standardizing numerical features such as `Age`, `Sleep Duration`, `Physical Activity Level`, `Blood Pressure`, `Heart Rate`, and `Daily Steps` is essential. This process ensures that variables with different units and scales contribute equally to the model, preventing biases and allowing machine learning algorithms to effectively learn patterns, leading to improved model performance.

Reference: https://www.kaggle.com/code/alnourabdalrahman9/sleeping-disorders-detection-outliers-removal#Splitting-the-Data:

### Dropping of unnecessary features

Some of the features in the dataset were deemed unnecessary and will be dropped/removed. 

In [ ]:
# Remove some features because they are unnecessary for analysis
features_to_remove = ['Person ID', 'Occupation', 'Quality of Sleep','Physical Activity Level','Sleep Disorder']

dataset.drop(features_to_remove, axis=1, inplace=True)
dataset.head()  # Verify that the features in the features_to_remove list have been dropped from the dataset